In [1]:
# Import libs
import os
import datetime
import tensorflow as tf
from tensorflow import keras

In [2]:
# Load dataset
# get data_dir for tranining
current_path = os.getcwd()

img_height = 224
img_width = 224
batch_size = 64

# set train dataset
train_data_dir = os.path.join(current_path, 'Vegetable_Images\\train')
train_ds = keras.utils.image_dataset_from_directory(
    train_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

# set validation dataset
val_data_dir = os.path.join(current_path, 'Vegetable_Images\\validation')
val_ds = keras.utils.image_dataset_from_directory(
    val_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

class_names = train_ds.class_names
class_num = len(class_names)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


In [3]:
# Normalize data
data_scaling = keras.layers.Rescaling(scale=1./255)

train_ds = train_ds.map( lambda x, y: (data_scaling(x), y))
val_ds = val_ds.map( lambda x, y: (data_scaling(x), y))

In [4]:
# Build model
MobileNet_excludeTop = keras.applications.MobileNet(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3)
)
MobileNet_excludeTop.trainable = False

Top_layers = keras.layers.GlobalAveragePooling2D()(MobileNet_excludeTop.output)
Top_layers = keras.layers.Dense(units=1024, activation='relu', use_bias=True)(Top_layers)
Top_layers = keras.layers.Dropout(rate=0.25)(Top_layers)
Top_layers = keras.layers.Dense(units=class_num, activation='softmax', use_bias=True)(Top_layers)

MobileNet_Model = keras.Model(inputs=MobileNet_excludeTop.input, outputs=Top_layers)

MobileNet_Model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

MobileNet_Model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 4,293,839 (16.38 MB)

 Trainable params: 1,064,975 (4.06 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [5]:
# Train model
logdir = os.path.join("logs", datetime.datetime.now().strftime("MobileNet_Model-%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = MobileNet_Model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tensorboard_callback]
)

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 121s 499ms/step - accuracy: 0.7532 - loss: 0.9150 - val_accuracy: 0.9930 - val_loss: 0.0512
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 123s 524ms/step - accuracy: 0.9905 - loss: 0.0530 - val_accuracy: 0.9963 - val_loss: 0.0242
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 124s 526ms/step - accuracy: 0.9968 - loss: 0.0256 - val_accuracy: 0.9970 - val_loss: 0.0153
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 124s 526ms/step - accuracy: 0.9984 - loss: 0.0138 - val_accuracy: 0.9977 - val_loss: 0.0106
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 123s 524ms/step - accuracy: 0.9990 - loss: 0.0090 - val_accuracy: 0.9977 - val_loss: 0.0089
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 123s 525ms/step - accuracy: 0.9995 - loss: 0.0060 - val_accuracy: 0.9980 - val_loss: 0.0071
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 123s 525ms/step - accuracy: 0.9995 - loss: 0.0052 - val_accuracy: 0.9973 - val_loss: 0.0066
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 123s 525ms/step - accuracy: 1.0000 -

In [6]:
# Save model as keras
MobileNet_Model.save('saved_models\MobileNet_Model.keras')